## play music of data


////////////////  TODO  ////////////////

- create specialist
- how to decide binary key-press? sample with probability og pressing X(beeing avg. keys pressed at any time?)
    - https://github.com/calclavia/DeepJ/blob/icsc/generate.py 
    - apply_temperature from code above makes it more likely to dont press buttons when there has been a pause
    - regularizing predicting the same note. by small negative
    - normalize predictionvectors before applying threshold
    - would be nice with teacher forcing, think it would be better with shortterm memory and rely less on "beat"
    - chunks probably work better to not make it so relyant on specific song and training on remembering.
    - could use scheduler to decrease learningrate after 150 epochs


In [ ]:
import sys
sys.path.append("helpers/")

#Libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F

print "torchversion: ", torch.__version__

#Custom libraries
import dataset
import datapreparation
from datapreparation import plot_loss
from train import train
from models import Generalist, Specialist

#PATHS
DATASET_PATH = 'datasets/training/'
FS1 = 'piano_roll_fs1/'
FS2 = 'piano_roll_fs2/'
FS5 = 'piano_roll_fs5/'
FULL_PATH = DATASET_PATH + FS5


%load_ext autoreload
%autoreload 2

# Inspect the dataset

In [ ]:

#LOAD
dataset_names = datapreparation.load_all_dataset_names(FULL_PATH)
print(dataset_names)
datasets = datapreparation.load_all_dataset(FULL_PATH)
#datasets_df = pd.DataFrame(datasets)

#CONSTANTS
max_length = datapreparation.get_max_length(datasets)
num_keys = datapreparation.get_numkeys(datasets)[0]
print("max_length: ", max_length)
print("num_keys: ", num_keys)

# Split datasets

In [ ]:
# shuffles the dataset on creation
Dataset = dataset.pianoroll_dataset_batch(FULL_PATH)
# splits the datasets
training_data, validation_data, testing_data = Dataset.split_datasets(split=[0.7,0.85, 1.0])


# Sample from dataset

In [ ]:
#VISUALIZE
print('Bach')
print('timesteps in song:', len(datasets[0][0]))
datapreparation.visualize_piano_roll(datasets[0])

datapreparation.embed_play_v1(datasets[0])



----
# Generalist composer
The aim is to train a neural network that can create music in general. It shall be kickstarted by real music. Meaning that it will get some seconds of real music and then it wil continue playing music.

In [ ]:
# set parameters for the model
input_size = num_keys
hidden_size = 64
num_tags = len(pd.unique(dataset_names))
num_layers = 1

# create model
generalist = Generalist(input_size, hidden_size, num_layers)

# set parameters for the training process
name = "generalist_BCE_1e-2"
epochs = 1000
batch_size = 4 # does not effect training. need to implement in def train
lr = 1e-2
criterion = None # it will create default criterion
optimizer = None # it will create default optimizer

In [ ]:
# TRAIN GENERALIST

trained_generalist, gen_train_loss, gen_val_loss, gen_train_acc, gen_test_acc  = train(generalist, training_data, validation_data, name, criterion, epochs, lr, optimizer, batch_size)


In [ ]:
# load loss
loaded_generalist_metrics = pickle.load(open( 'saved_losses/generalist_BCE_1e-2_999.pickle', "rb" ))
plot_loss(loaded_generalist_metrics)

In [ ]:
# generate song from generalist

# pick song
#song = training_data[10]
song = testing_data[1]
# load best model
model = Generalist(input_size, hidden_size, num_layers) # need to know input_size, hidden_size excetera.. :/
model.load_state_dict(torch.load('saved_models/generalist_BCE_1e-2_208.pth')) #best so far on song test:1, train:10 with init=10 not good now
#model.load_state_dict(torch.load('saved_models/generalist_BCE_1e-2_175.pth'))

In [ ]:
datapreparation.gen_music_seconds(model,init=song[0],composer=song[1],fs=5,gen_seconds=60,init_seconds=20)

----
# Specialist composer
The aim is to train a neural network that can create music similar to a specific composer. It shall be kickstarted by a song from that composer. Meaning that it will get some seconds of the song and then it should be able to compose music that is typically similar to that specific composer

In [ ]:
# set parameters for the model
input_size = num_keys
hidden_size = 64
num_tags = len(pd.unique(dataset_names))
num_layers = 1

# create model
specialist = Specialist(input_size, hidden_size, num_layers)

# set parameters for the training process
name = "Specialist_BCE_1e-2"
epochs = 1000
batch_size = 4 # does not effect training. need to implement in def train
lr = 1e-2
criterion = None # it will create default criterion
optimizer = None # it will create default optimizer


In [ ]:
# TRAIN SPECIALIST

trained_specialist, spe_train_loss, spe_val_loss, spe_train_acc, spe_test_acc  = train(specialist, training_data, validation_data, name, criterion, epochs, lr, optimizer, batch_size)


In [ ]:
# load loss
loaded_specialist_metrics = pickle.load(open( 'saved_losses/specialist_BCE_1e-2_999.pickle', "rb" ))
plot_loss(loaded_specialist_metrics)


In [ ]:
# generate song from specialist

# pick song
#song = training_data[10]
song = testing_data[1]

# load best model
model = Specialist(input_size, hidden_size, num_layers) # need to know input_size, hidden_size excetera.. :/
model.load_state_dict(torch.load('saved_models/specialist_BCE_1e-2_208.pth')) 
    

In [ ]:
# generate
datapreparation.gen_music_seconds(model,init=song[0],composer=song[1],fs=5,gen_seconds=60,init_seconds=20)

# Comments
- specializes on one song because of small sample size, but huge samples
    

# Further works
issue 1: specializes very quickly
- bigger dataset which there is here: http://www.piano-midi.de/
- chunks should maybe remove some of that effect

